In [1]:
import pandas as pd
journal_entries_2020 = pd.read_excel("Vinatics 2020 Journal Entries.xlsx")
trial_balance_2020 = pd.read_excel("Vinatics 2020 Trial Balance.xlsx")

In [2]:
journal_entries_2020.head()

,JE number,GL Code,Period,Effective Date,Entry Date,Source,Source_Group,Functional Amount,Preparer ID
0,210000003,60000001,2020-001,2020-01-22,2020-01-22,Invoice,Purchases,-414581.31680,Merrie Beaver
1,210000003,60000112,2020-001,2020-01-22,2020-01-22,Invoice,Purchases,12504.18092,Merrie Beaver
2,210000003,60000112,2020-001,2020-01-22,2020-01-22,Invoice,Purchases,14710.86008,Merrie Beaver
3,210000003,60000112,2020-001,2020-01-22,2020-01-22,Invoice,Purchases,285390.59640,Merrie Beaver
4,210000003,60000112,2020-001,2020-01-22,2020-01-22,Invoice,Purchases,12504.23664,Merrie Beaver


In [3]:
pivot_2020_journals = journal_entries_2020.pivot_table(index = "GL Code", values = "Functional Amount", aggfunc=sum)
pivot_2020_journals = pd.DataFrame(pivot_2020_journals.to_records())
pivot_2020_journals.head()

,GL Code,Functional Amount
0,20000000,4.328652e+06
1,20000002,-1.634499e+05
2,20000003,-1.929373e+06
3,20000004,-6.265112e+05
4,20000005,-9.460914e+05


In [4]:
trial_balance_2020.head()

,Account type,Account Class,Account Code,Account name,Opening Balance,Closing Balance
0,Assets,Receivables,20000050,Customer invoices,3.945206e+05,8.145908e+04
1,Assets,Receivables,20000051,Credit notes,0.000000e+00,0.000000e+00
2,Assets,Receivables,20000000,Receivables 1,5.439532e+07,5.872397e+07
3,Assets,Receivables,20000021,Receivables 2,1.038470e+04,8.362068e+03
4,Assets,Receivables,20000002,Receivables 3,-2.898225e+04,-1.924322e+05


In [5]:
tb_2020 = trial_balance_2020
tb_2020 = tb_2020.merge(pivot_2020_journals, how = "left", left_on="Account Code", right_on="GL Code").drop("GL Code", axis = 1)
tb_2020.head()

,Account type,Account Class,Account Code,Account name,Opening Balance,Closing Balance,Functional Amount
0,Assets,Receivables,20000050,Customer invoices,3.945206e+05,8.145908e+04,-3.130615e+05
1,Assets,Receivables,20000051,Credit notes,0.000000e+00,0.000000e+00,NaN
2,Assets,Receivables,20000000,Receivables 1,5.439532e+07,5.872397e+07,4.328652e+06
3,Assets,Receivables,20000021,Receivables 2,1.038470e+04,8.362068e+03,-2.022636e+03
4,Assets,Receivables,20000002,Receivables 3,-2.898225e+04,-1.924322e+05,-1.634499e+05


In [6]:
tb_2020 = tb_2020.fillna(0)
tb_2020.head()

,Account type,Account Class,Account Code,Account name,Opening Balance,Closing Balance,Functional Amount
0,Assets,Receivables,20000050,Customer invoices,3.945206e+05,8.145908e+04,-3.130615e+05
1,Assets,Receivables,20000051,Credit notes,0.000000e+00,0.000000e+00,0.000000e+00
2,Assets,Receivables,20000000,Receivables 1,5.439532e+07,5.872397e+07,4.328652e+06
3,Assets,Receivables,20000021,Receivables 2,1.038470e+04,8.362068e+03,-2.022636e+03
4,Assets,Receivables,20000002,Receivables 3,-2.898225e+04,-1.924322e+05,-1.634499e+05


In [7]:
tb_2020['Calculated Closing Balance'] = tb_2020['Opening Balance'] + tb_2020['Functional Amount']
tb_2020['Difference'] = tb_2020['Closing Balance'] - tb_2020['Calculated Closing Balance']
tb_2020_diff = tb_2020.loc[tb_2020['Difference'] != 0]
tb_2020_diff

,Account type,Account Class,Account Code,Account name,Opening Balance,Closing Balance,Functional Amount,Calculated Closing Balance,Difference
163,Assets,Receivables,20001101,VAT sales,4.225803e-07,2.818031e-07,-1.445026e-07,2.780778e-07,3.725290e-09
249,Revenue,Revenue from contract with customers,120010010,Domestic sales,5.961845e+06,9.790603e+06,3.828757e+06,9.790603e+06,-1.862645e-09
310,Expenses,Inventories,140203004,Packaging,2.613874e+08,4.171158e+08,1.557284e+08,4.171158e+08,5.960464e-08


In [8]:
tb_2020_diff.to_csv('Journal Entry Completeness.csv')

In [ ]:
import pandas as pd
journal_entries_2020 = pd.read_excel("Vinatics 2020 Journal Entries.xlsx")
trial_balance_2020 = pd.read_excel("Vinatics 2020 Trial Balance.xlsx")
pivot_2020_journals = journal_entries_2020.pivot_table(index = "GL Code", values = "Functional Amount", aggfunc=sum)
pivot_2020_journals = pd.DataFrame(pivot_2020_journals.to_records())
tb_2020 = trial_balance_2020
tb_2020 = tb_2020.merge(pivot_2020_journals, how = "left", left_on="Account Code", right_on="GL Code").drop("GL Code", axis = 1)
tb_2020 = tb_2020.fillna(0)
tb_2020['Calculated Closing Balance'] = tb_2020['Opening Balance'] + tb_2020['Functional Amount']
tb_2020['Difference'] = tb_2020['Closing Balance'] - tb_2020['Calculated Closing Balance']
tb_2020_diff = tb_2020.loc[tb_2020['Difference'] != 0]
tb_2020_diff
tb_2020_diff.to_csv('Journal Entry Completeness.csv')